# 🔗 Dataset: Pima Indians Diabetes

*Small and ideal for logistic regression / binary classification.*

In [122]:
import torch
import torch.optim as optim

In [ ]:
import pandas as pd

url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"

df = pd.read_csv(url, header=None)
print(df.head())

   0    1   2   3    4     5      6   7  8
0  6  148  72  35    0  33.6  0.627  50  1
1  1   85  66  29    0  26.6  0.351  31  0
2  8  183  64   0    0  23.3  0.672  32  1
3  1   89  66  23   94  28.1  0.167  21  0
4  0  137  40  35  168  43.1  2.288  33  1


In [124]:
X = df.iloc[:, :-1].values   # first 8 columns → features
y = df.iloc[:, -1].values   # last column → label (0 or 1)

In [125]:
X

array([[  6.   , 148.   ,  72.   , ...,  33.6  ,   0.627,  50.   ],
       [  1.   ,  85.   ,  66.   , ...,  26.6  ,   0.351,  31.   ],
       [  8.   , 183.   ,  64.   , ...,  23.3  ,   0.672,  32.   ],
       ...,
       [  5.   , 121.   ,  72.   , ...,  26.2  ,   0.245,  30.   ],
       [  1.   , 126.   ,  60.   , ...,  30.1  ,   0.349,  47.   ],
       [  1.   ,  93.   ,  70.   , ...,  30.4  ,   0.315,  23.   ]])

In [126]:
X.shape

(768, 8)

In [127]:
X = torch.from_numpy(X).float()
y = torch.from_numpy(y).float()

In [128]:
X


tensor([[  6.0000, 148.0000,  72.0000,  ...,  33.6000,   0.6270,  50.0000],
        [  1.0000,  85.0000,  66.0000,  ...,  26.6000,   0.3510,  31.0000],
        [  8.0000, 183.0000,  64.0000,  ...,  23.3000,   0.6720,  32.0000],
        ...,
        [  5.0000, 121.0000,  72.0000,  ...,  26.2000,   0.2450,  30.0000],
        [  1.0000, 126.0000,  60.0000,  ...,  30.1000,   0.3490,  47.0000],
        [  1.0000,  93.0000,  70.0000,  ...,  30.4000,   0.3150,  23.0000]])

In [129]:
# Normalize inputs (very important here)
# X = (X - X.mean(dim=0))/X.std(dim=0)
X = X / X.max(dim=0).values

In [130]:
X

tensor([[0.3529, 0.7437, 0.5902,  ..., 0.5007, 0.2591, 0.6173],
        [0.0588, 0.4271, 0.5410,  ..., 0.3964, 0.1450, 0.3827],
        [0.4706, 0.9196, 0.5246,  ..., 0.3472, 0.2777, 0.3951],
        ...,
        [0.2941, 0.6080, 0.5902,  ..., 0.3905, 0.1012, 0.3704],
        [0.0588, 0.6332, 0.4918,  ..., 0.4486, 0.1442, 0.5802],
        [0.0588, 0.4673, 0.5738,  ..., 0.4531, 0.1302, 0.2840]])

In [131]:
X.shape

torch.Size([768, 8])

In [132]:
class Linear:
    def __init__(self, n_input, n_neurons):
        self.weights = 0.01 * (torch.rand(n_input, n_neurons, dtype=torch.float32))
        self.weights.requires_grad_()
        self.bias = torch.zeros((1, n_neurons), dtype=torch.float32)
        self.bias.requires_grad_()

    def forward(self, input):
        self.output = torch.matmul(input, self.weights) + self.bias

In [133]:
class Activation_Relu:
    def forward(self, input):
        self.output = torch.maximum(input, torch.tensor(0.0, device=input.device))

In [134]:
class Sigmoid:
    def forward(self, input):
        self.output = 1 / (1 + torch.exp(-input))

# Binary Cross-Entropy Loss from scratch

*Inputs: y_pred must be probabilities (after sigmoid), y_true must be 0/1 floats.*

In [135]:
class Loss_BinaryCrossEntropy:
    def forward(self, y_pred, y_true):
         # clamp to avoid log(0)
          y_pred = torch.clamp(y_pred, 1e-7, 1 - 1e-7)
          # BCE formula: -( y*log(p) + (1-y)*log(1-p) )
          loss = - (y_true*torch.log(y_pred) + (1 - y_true) * torch.log(1 - y_pred))
          return torch.mean(loss)

In [136]:
linear1 = Linear(8, 16)
activation1 = Activation_Relu()
linear2 = Linear(16, 8)
activation2 = Activation_Relu()
linear3 = Linear(8,4)
activation3 = Activation_Relu()
output_layer = Linear(4,1)
sigmoid = Sigmoid()
LBC = Loss_BinaryCrossEntropy()

# linear1.forward(X)
# activation1.forward(linear1.output)

# linear2.forward(activation1.output)
# activation2.forward(linear2.output)

# linear3.forward(activation2.output)
# activation3.forward(linear3.output)

# output_layer.forward(activation3.output)
# sigmoid.forward(output_layer.output)
# print(sigmoid.output[:10])

# Collect parameters (this is REQUIRED)

*Because you built everything from scratch, PyTorch does not know what to update unless you tell it.*

In [137]:
params = [
    linear1.weights, linear1.bias,
    linear2.weights, linear2.bias,
    linear3.weights, linear3.bias,
    output_layer.weights, output_layer.bias
]

In [138]:
# Create optimizer (SGD or Adam)
optimizer = torch.optim.SGD(params, lr=0.1)

In [140]:
epochs = 100
batch_size = 32
n_samples = X.shape[0]

for epoch in range(epochs):

    # shuffle data every epoch
    indices = torch.randperm(n_samples)

    epoch_loss = 0.0
    num_batches = 0

    for start in range(0, n_samples, batch_size):
        batch_idx = indices[start:start + batch_size]

        Xb = X[batch_idx]          # (B, 8)
        yb = y[batch_idx]          # (B, 1)

        # 1️⃣ clear previous gradients
        optimizer.zero_grad()

        # -------- forward pass (YOUR CODE) --------
        linear1.forward(Xb)
        activation1.forward(linear1.output)

        linear2.forward(activation1.output)
        activation2.forward(linear2.output)

        linear3.forward(activation2.output)
        activation3.forward(linear3.output)

        output_layer.forward(activation3.output)
        sigmoid.forward(output_layer.output)

        loss = LBC.forward(sigmoid.output, yb)
        # -----------------------------------------

        # 2️⃣ backpropagation (AUTOGRAD)
        loss.backward()

        # 3️⃣ update weights
        optimizer.step()

        epoch_loss += loss.item()
        num_batches += 1

    avg_loss = epoch_loss / num_batches

    # quick accuracy check (optional)
    preds = (sigmoid.output >= 0.5).float()
    acc = (preds == yb).float().mean().item()

    print(f"Epoch {epoch+1}/{epochs}  Loss: {avg_loss:.4f}  Acc(last batch): {acc:.3f}")


Epoch 1/100  Loss: 0.6472  Acc(last batch): 0.594
Epoch 2/100  Loss: 0.6471  Acc(last batch): 0.656
Epoch 3/100  Loss: 0.6473  Acc(last batch): 0.844
Epoch 4/100  Loss: 0.6471  Acc(last batch): 0.750
Epoch 5/100  Loss: 0.6471  Acc(last batch): 0.625
Epoch 6/100  Loss: 0.6472  Acc(last batch): 0.781
Epoch 7/100  Loss: 0.6470  Acc(last batch): 0.750
Epoch 8/100  Loss: 0.6471  Acc(last batch): 0.719
Epoch 9/100  Loss: 0.6471  Acc(last batch): 0.500
Epoch 10/100  Loss: 0.6471  Acc(last batch): 0.625
Epoch 11/100  Loss: 0.6471  Acc(last batch): 0.625
Epoch 12/100  Loss: 0.6471  Acc(last batch): 0.594
Epoch 13/100  Loss: 0.6472  Acc(last batch): 0.719
Epoch 14/100  Loss: 0.6471  Acc(last batch): 0.656
Epoch 15/100  Loss: 0.6471  Acc(last batch): 0.750
Epoch 16/100  Loss: 0.6474  Acc(last batch): 0.812
Epoch 17/100  Loss: 0.6472  Acc(last batch): 0.656
Epoch 18/100  Loss: 0.6471  Acc(last batch): 0.656
Epoch 19/100  Loss: 0.6472  Acc(last batch): 0.562
Epoch 20/100  Loss: 0.6470  Acc(last bat